## concurrent downloading

Sometimes, it takes long time to download the huge website if download them sequentially, however, if we want to minimize the time spent there, concurrent download is the best choice.

1. one million web pages

2. sequential crawler

3. threaded crawler

4. multiprocessing crawler


### one million web pages

the one million web pages have already been given as a csv file,here we will extract them from the zip file


In [2]:
import csv
from zipfile import ZipFile
from io import BytesIO, TextIOWrapper
import requests

In [3]:
resp = requests.get('http://s3.amazonaws.com/alexa-static/top-1m.csv.zip', stream=True)

In [4]:
urls = [] # top 1 million URL's will be stored in this list 
with ZipFile(BytesIO(resp.content)) as zf:
    csv_filename = zf.namelist()[0]
    with zf.open(csv_filename) as csv_file:
        for _, website in csv.reader(TextIOWrapper(csv_file)):
            urls.append('http://' + website)

In [5]:
class AlexaCallback:
    def __init__(self, max_urls=500):
        self.max_urls = max_urls
        self.seed_url = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
        self.urls = []

    def __call__(self):
        resp = requests.get(self.seed_url, stream=True)
        with ZipFile(BytesIO(resp.content)) as zf:
            csv_filename = zf.namelist()[0]
            with zf.open(csv_filename) as csv_file:
                for _, website in csv.reader(TextIOWrapper(csv_file)):
                    self.urls.append('http://' + website)
                    if len(self.urls) == self.max_urls:
                        break

### Threaded crawler

explanation : 

Implementing a multithreaded crawler